In [16]:
import cv2 #opencv
import numpy as np
import os   #helps with path
from matplotlib import pyplot as plt #to use plt.imshow()
import time              #to measure time between frames 
import mediapipe as mp   
mpDraw=mp.solutions.drawing_utils
mpHands=mp.solutions.hands
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB because by default opencv use bgr but we need rgb for mediapipe to process image
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            mpDraw.draw_landmarks(image,handsLms, mpHands.HAND_CONNECTIONS,
                             mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) )
def get_label(index,results):
    label = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            label = classification.classification[0].label
#         print(index,idx,label)
    if label:
        return label
    else:
        if index == 1:
            return get_label(0,results)
        elif index == 0:
            return get_label(1,results)
        else:
            return label
def extract_keypoints(results):
    hands = [np.zeros(21*3),np.zeros(21*3)]
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            label = get_label(num,results)
            if label == 'Right':
                hands[0] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            if label == 'Left':
                hands[1] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            
    return np.concatenate(hands)
categories = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','space','nothing'])
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
ACCURACY_THRESHOLD = 0.97
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # print(logs.get('acc'))
        # print(logs.get('categorical_accuracy'))
        if(logs.get('categorical_accuracy') > ACCURACY_THRESHOLD):
            # print(logs.get('acc'))
            # print(logs.get('categorical_accuracy'))
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()
log_dir = os.path.join('trainLogsdNeuralNetworkATOZ26April')
tb_callback = TensorBoard(log_dir=log_dir)
mycallbacks = [callbacks,tb_callback]
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(15,126)))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(categories.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('model26April.h5')

In [17]:
def prob_viz(res, actions,input_frame):
    output_frame = input_frame.copy()
    if res[np.argmax(res)] > 0.8:
        cv2.putText(output_frame,actions[np.argmax(res)], (200,300), cv2.FONT_HERSHEY_SIMPLEX,4, (0, 0, 255), 2, cv2.LINE_AA)
    return output_frame

In [18]:
back = cv2.imread('board1.jpg')
back = cv2.resize(back,(600,600))

In [19]:
categoriesLatest = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','d',' ','n'])

In [28]:
import time
sequence = []
sentence = []
threshold = 0.8
time_frame = []
cap = cv2.VideoCapture(0)
started = False
# Set mediapipe model 
with mpHands.Hands() as hand:
    i = 20
    j = 26
    back2 = cv2.imread('board1.jpg')
    back2 = cv2.resize(back,(600,600))
    while cap.isOpened():
        
        board = back
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame,hand)

        
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
#         print(sequence)
        sequence = sequence[-15:]
#         print(sequence)
        
        if len(sequence) == 15:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                casl = categoriesLatest[np.argmax(res)]
                if casl != 'n' and (cv2.waitKey(1) & 0xFF == ord('v')) :
                    cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    if i <= 20 and  started:
                        j = j + 26
                    i = (i + 20)%500
                    if i == 0:
                        i = 20
                    started = True

            # Viz probabilities
            board = prob_viz(res, categories,board)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.imshow('viz',board)
        cv2.imshow('Text Converter',back2)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# To print you have to press v and to quit press q multiple times